In [1]:
import sqlite3
import pandas as pd

In [2]:
db = sqlite3.connect('testdb.db')
cur = db.cursor()

## Column constraints


- PRIMARY KEY = NOT NULL + UNIQUE
- NOT NULL
- UNIQUE
- FOREIGN KEY

In [5]:
cur.execute("""CREATE TABLE IF NOT EXISTS store(
id integer PRIMARY KEY,
name text) """)


cur.execute("""CREATE TABLE IF NOT EXISTS sales(
id integer PRIMARY KEY,
store_id integer,
FOREIGN KEY (store_id) REFERENCES store (id))""")



In [8]:
cur.execute("""INSERT INTO store VALUES(2, 'Ica') """)
db.commit()

## pandas

Vi kan använda pandas för att ladda in data från en relationsdatabas med SQL. Detta kan vara nödvändigt när man måste koppla upp sig till en stor databas där mäbngden data är för stor för att ha i datorns RAM-minne.

Vi börjar med att spara ner en dataframe till en databas som vi sedan kan använda för att testa detta

In [2]:
weather = pd.read_csv('austin_weather.csv')
weather.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,30.33,30.27,10,10,7,10,2,16,T,


In [3]:
db = sqlite3.connect('austin_weather.db')
weather.to_sql('weather', db) #weather blir nament på tabellen i datbasen

## Nu kan vi läsa in data till en dataframe från databasen

syntax: df = pd.read_sql(sql_anrop, db_connection)

In [4]:
df = pd.read_sql("SELECT Date, TempHighF, TempLowF FROM weather", db)
df.head()

,Date,TempHighF,TempLowF
0,2013-12-21,74,45
1,2013-12-22,56,39
2,2013-12-23,58,32
3,2013-12-24,61,31
4,2013-12-25,58,41


In [5]:
df = pd.read_sql("SELECT * FROM weather", db)
df.head()

,index,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,0,2013-12-21,74,60,45,67,49,43,93,75,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,1,2013-12-22,56,48,39,43,36,28,93,68,...,30.13,29.87,10,10,5,16,6,25,0,
2,2,2013-12-23,58,45,32,31,27,23,76,52,...,30.49,30.41,10,10,10,8,3,12,0,
3,3,2013-12-24,61,46,31,36,28,21,89,56,...,30.45,30.3,10,10,7,12,4,20,0,
4,4,2013-12-25,58,50,41,44,40,36,86,71,...,30.33,30.27,10,10,7,10,2,16,T,


## Vi vill dra ut alla rader med WindHighMPH > 10 från databasen

In [34]:
df = pd.read_sql("SELECT Date, WindHighMPH, TempHighF FROM weather WHERE WindHighMPH > 10", db)
df.head()

,Date,WindHighMPH,TempHighF
0,2013-12-21,20,74
1,2013-12-22,16,56
2,2013-12-23,8,58
3,2013-12-24,12,61
4,2013-12-26,12,57


#### Det funkade inte. Vi undersöker varför...

In [35]:
df.dtypes

Date           object
WindHighMPH    object
TempHighF       int64
dtype: object

kolumnen WindHighMPH är av typen object och därför har "WHERE WindHighMPH > 10" inte fungerat. För att få ut datan vi vill ha måste vi casta kolumnen till int... 

I vår kolumn hade vi minst ett värde som var '-' och kunde därför inte casta hela kolumnen direkt. Det finns två alternativ för att komma runt detta

#### Alt 1 - boolean masking som specifikt kollar på '-'

In [36]:
df = df[df['WindHighMPH'] != '-' ]


#### Alt 2 - boolean masking som använder apply och en lambda funktion för att kolla om kolumnvärden är digits

In [37]:
df2 = df[ df['WindHighMPH'].apply(lambda x: str(x).isdigit())]

## Casta kolumnen till int

In [38]:
df['WindHighMPH'] = df['WindHighMPH'].astype(int)

In [39]:
df.dtypes

1177

## nu kan vi använda pandas för att få ut de dagar som hade WindHighMPH > 10

In [40]:
print(len(df)) # Sanity check
subset_df = df[ df['WindHighMPH'] > 10]
len(subset_df)

1177


977

## Bonus! Visualization med altair

Altair är ett bibliotek jag nyligen upptäckt för visualisering som verkar väldigt kraftfullt utan mycket customization

Prova att installera det och kolla in docs på https://altair-viz.github.io/getting_started/starting.html#the-data

In [56]:
!pip install altair

### En enkel lineplot med altair

In [44]:
import altair as alt

data = df[df.Date > '2017-01-01']

alt.Chart(data).mark_line().encode(
    x = 'Date',
    y='TempHighF')

alt.Chart(...)

### Genom att byta ut mark_line mot mark_point får vi en scatterplot istället

In [45]:
alt.Chart(data).mark_point().encode(
    x = 'Date',
    y='TempHighF')

alt.Chart(...)